<a href="https://colab.research.google.com/github/tefarov/netology_pyda5/blob/master/homework_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import sys

# Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [0]:
STR_FILEMOV = sys.path[0] + '\\movies.csv'
STR_FILERAT = sys.path[0] + '\\ratings.csv'

def fx_class(rating):
  "This will create human-readable rating-names"
  if not rating:
      return np.nan
  elif not type(rating) == float:
      return 'invalid type'
  elif rating <= 2:
      return 'низкий рейтинг'
  elif rating <= 4:
      return 'средний рейтинг'
  elif rating <= 5:
      return 'высокий рейтинг'
    
  return '??'

# Merge 2 datasets just to see the names of the movies being rated
print('Reading: ', STR_FILEMOV)
mov = pd.read_csv(STR_FILEMOV)
print('Reading: ', STR_FILERAT)
rat = pd.read_csv(STR_FILERAT)
rat = rat.merge(mov,on='movieId',how='left')

# apply data classification
rat['class'] = rat['rating'].apply(fx_class)

# display results
print(rat.head(100))
print(rat.info())

# Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
```
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

```
Результат классификации запишите в отдельный столбец region.

In [0]:
STR_FILEPTH = sys.path[0] + '\\keywords.csv'

geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

def fx_geoclass(text):
    if not text: return 'undefined'
    if not type(text) == str: text=str(text)
    
    for reg,cities in geo_data.items():
        for cit in cities:
            if cit in text:
                return reg
    
    return 'undefined'

print('Reading: ', STR_FILEPTH)
dat = pd.read_csv(STR_FILEPTH)

dat['region'] = dat['keyword'].apply(fx_geoclass)
dat = dat[dat['region'] != 'undefined']

for ix,rec in dat[:100].iterrows():
    print("{1:>25} -- '{0}'".format(rec['keyword'],rec['region']))



# Задание 3
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.
При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:
## 1.
В переменную years запишите список из всех годов с 1950 по 2010.
## 2.
Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
1.  для каждой строки пройдите по всем годам списка years
2.  если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
3.  если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

## 3.
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’
## 4.
Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга


In [0]:
STR_FILEMOV = sys.path[0] + '\\movies.csv'
STR_FILERAT = sys.path[0] + '\\ratings.csv'
   
print('Reading: ', STR_FILEMOV)
mov = pd.read_csv(STR_FILEMOV)
print('Reading: ', STR_FILERAT)
rat = pd.read_csv(STR_FILERAT)
print('-'*10)

# Извлекаем год из названия, которое оканчивается годом в скобках
# например: Title ({YEAR})EOL
# на тот случай, если в названии фильма указан какой-то год
mov['year'] = mov['title'].str.extract(r'((?<=\()\d{4}(?=\)\s*$))',expand=False)

# хотя некоторые названия не подходят под шаблон,
# можно было бы применить ещё пару шаблонов, но
# давайте просто выдернем любой год из названия фильма

# чтобы не обрабатывать обработанные строки ещё раз
# возьмём набор данных с только необработанными годами
# обработаем их и сохраним в исходный набор
upd = mov[mov['year'].isna()].copy()
upd['year'] = upd['title'].str.extract(r'(\d{4})',expand=False)
upd.fillna('1900',inplace=True)
mov.update(upd[['year']],overwrite=False) # сохраняем в исходный набор

# теперь когда мы определили года, давайте определим десятилетия
# 199x для 1990..1999, 200x для 2000..2009 ..  итд
mov['decade'] = mov['year'].apply(lambda x:x[:3] + '0-е')

# ГОДА и ДЕСЯТИЛЕТИЯ определены, переходим к статистике

# сливаем наборы рэйтинга и кинофильмов
rat = rat.merge(mov,on='movieId',how='left')

# обработаем среднее по годам
grp = rat.groupby('year').mean()[['rating']]
grp = grp.sort_values(by=['rating'],ascending=[False])
for ix,rec in grp.iterrows():
    print("{0: >4} -- {1:.2f}".format(ix,rec['rating']))
print('-'*10)

# обработаем среднее по десятилетиям
grp = rat.groupby('decade').mean()[['rating']]
grp = grp.sort_values(by=['rating'],ascending=[False])
for ix,rec in grp.iterrows():
    print("{0: >4} -- {1:.2f}".format(ix,rec['rating']))
print('-'*10)

# ПОБЕДА !!
# но вообще анализ конечно весьма такой поверхностный